In [ ]:
%pip install git+https://github.com/cioos-sioocean-data-parser.git

# Parse Amundsen INT to NetCDF

In [ ]:
from ocean_data_parser.parsers import amundsen, utils
from ocean_data_parser.metadata import pdc
from glob import glob 
import logging
import re

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger()

# Get the list of INT files

In [2]:
PATH_TO_INT = "./../tests/parsers_test_files/amundsen/"
search_path = f"{PATH_TO_INT}/**/*.int"
int_files = glob(search_path,recursive=True)
print(f"{len(int_files)} int files are available within:\n{search_path}")

110 int files are available within:
./../tests/parsers_test_files/amundsen//**/*.int


# Convert INT files to NetCDF equivalent 

In [10]:

# Loop through each INT detected
for file in int_files:
    # Parse seabird 
    ds = amundsen.int_format(file)
    if ds is None:
        continue 

    # Get global attribute based on CCIN
    ccin = re.search('amundsen/(\d+)',file)
    if ccin:
        ds.attrs = pdc.fgdc_to_acdd(url=f"https://www.polardata.ca/pdcsearch/xml/fgdc/{ccin[1]}_fgdc.xml")
    else:
        logger.warning("Can't detect CCIN")
    
    # Clean up dataset and fix common issues
    ds = utils.standardize_dateset(ds)
    ds.to_netcdf(f"{file}_test.nc")